# Разница между GET- и POST-запросами

__Напоминание__:

При работе с вебом HTTP-запросы используются для взаимодействия с серверами. Два наиболее часто применяемых метода — GET и POST. Они определяют, как клиент (браузер, скрипт, API-запрос) запрашивает или отправляет данные на сервер.

### Основные различия между GET и POST

| Характеристика   | GET                                      | POST                                       |
|-----------------|-----------------------------------------|-------------------------------------------|
| **Цель**        | Получение данных с сервера              | Отправка данных на сервер                 |
| **Передача данных** | В URL (через `?key=value`)            | В теле запроса (через `data={}` или `json={}`) |
| **Безопасность** | Менее безопасный (виден в URL)         | Безопаснее (данные скрыты в теле запроса) |
| **Кэширование**  | Может кешироваться браузером          | Не кешируется                             |
| **Использование** | Получение веб-страниц, API-запросы на чтение | Авторизация, отправка форм, загрузка файлов |


#### Пример 1: GET-запрос работает без дополнительных заголовков

Некоторые сайты требуют наличия определённых HTTP-заголовков, таких как `User-Agent`, или же устанавливают cookies, которые используются для аутентификации или ограничения запросов. В таких случаях обычный `requests.get()` может не дать ожидаемый результат.

В примере ниже всё работает без "танцев с бубном" (a.k.a. дополнительных заголовков или установок куки)

In [179]:
import requests

URL = "https://student.itmo.ru/ru/dormitory/"
response = requests.get(URL)

# Выводим статус
print(f"Статус: {response.status_code}")
# Выводим кодировку (на всякий случай)
print(f"Кодировка: {response.encoding}")

Статус: 200
Кодировка: UTF-8


In [183]:
import webbrowser

with open("itmo_dorms.html", "w", encoding="utf-8") as f:
    f.write(response.text)

webbrowser.open("itmo_dorms.html")

True

#### Пример 2: GET-запрос (когда нужны куки)

Некоторые сайты требуют передачи User-Agent или cookies для успешного выполнения запроса. Например, при попытке извлечения страниц с сервиса Кинопоиск без cookies можно столкнуться с проблемами:

<img src="cookies.jpg" width=500 height=500/, title="демотиватор">

In [184]:
import requests

URL = "https://www.kinopoisk.ru/lists/movies/genre--thriller"
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0"}
param_dict = {"page": "1"}
response = requests.get(url=URL, headers=headers, params=param_dict)

Запрос выше вернёт страницу, которая содержит запрос на ввод капчи от клиента (как часть анти-бот системы сайта)

Объект `Session()` в библиотеке requests позволяет сохранять состояние между запросами:
- Автоматически сохраняет и передаёт cookies (главное)
- Позволяет переиспользовать заголовки (headers), параметры (params) и другие настройки (удобно)

Как `Session()` помогает обходить ограничения?
- С ответом на первый запрос сервер может отправить cookies, которые браузер обычно сохраняет
- При повторных запросах браузер автоматически отправляет сохранённые cookies
- `requests.Session()` позволяет имитировать такое поведение, автоматически запоминая и отправляя cookies

---


Иными словами, использование `Session()` оправдано, когда (хотя бы одно):
- Делается много однотипных запросов к подкаталогам одного сайта
- Хочется хранить информацию об одном и том же наборе cookies между запросами, не передавая их явно каждый раз
- Сайт требует авторизации (например, через POST-запрос с логином и паролем), про это ниже

Можно обойтись без `Session()`, если:
- Запросов мало, и они не требуют сохранения cookies
- Сайт не использует cookies для проверки пользователя

**Примечание** В некоторых случаях получение таким способом cookies позволяет лишить нас необходимости эмулирования браузерной сесссии - т.е. попытки закосить под реальный браузерный запрос (в этом смысле он может выступать альтернативой для Селениума [про него ниже]), однако **это не работает всегда**; хотя в случае с кинопоиском, кажется, работает :)

In [192]:
import requests

URL = "https://www.kinopoisk.ru/lists/movies/genre--thriller"

# Создаём сессию
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0"
})

# Посмотрим, какие cookies установлены перед первым запросом
print("Cookies перед запросом:", session.cookies)
print('\n')

# Первый запрос получит cookies
response_1 = session.get(URL)
print("Cookies после первого запроса:", session.cookies)
print('\n')

# Второй запрос уже будет отправлен с полученными cookies
response_2 = session.get(URL)

Cookies перед запросом: <RequestsCookieJar[]>


Cookies после первого запроса: <RequestsCookieJar[<Cookie _yasc=5JgFbIU3HuC6jZHlPO7ym5yisddjskmi5bQcP0m/upevhrkGOqG7az4OIVnSLuvcMw== for .kinopoisk.ru/>, <Cookie disable_server_sso_redirect=1 for .kinopoisk.ru/>, <Cookie i=8a+qKhZ8T+Q4g+VL4FMedyUsmpRZcSCHY70dEDvrJbWKbDKFPdYC4r7eFaaSNdiFw/qFqGMu/ikfawA6nLd24OlXSFg= for .kinopoisk.ru/>, <Cookie yandexuid=7622292021741000088 for .kinopoisk.ru/>, <Cookie yashr=6582575981741000088 for .kinopoisk.ru/>, <Cookie mda2_beacon=1741000089092 for .passport.yandex.ru/>, <Cookie mda2_domains=kinopoisk.ru for .passport.yandex.ru/>, <Cookie i=jyztl+WUQSAYHDQFewB8BK/XZlS4I48bJFhnZQAxv0ZB6Ok0QVV2A3OAUj/yBQyqFoM2ScEZ554X9XhPtgAuyeegWL8= for .yandex.ru/>, <Cookie yandexuid=8235799261741000089 for .yandex.ru/>, <Cookie ys=c_chck.1663399360 for .yandex.ru/>, <Cookie _csrf=BQO4bz-t5jgzKQqOhG9nlC5q for www.kinopoisk.ru/>]>




In [195]:
with open("kinopoisk.html", "w", encoding="utf-8") as f:
    f.write(response_2.text)


import webbrowser
webbrowser.open("kinopoisk.html")

True

Про объект `Session()` можно почитать:
- [Здесь](https://docs-python.ru/packages/modul-requests-python/sessii-seansy-session/)
- [В документации](https://requests.readthedocs.io/en/latest/user/advanced/)

В веб-скрэппинге `POST`-запросы чаще всего используются в двух случаях:

**Авторизация на сайте**  
   - Многие сайты требуют логин и пароль для доступа к данным.
   - `POST`-запрос отправляет учетные данные на сервер, который возвращает токен или сессию (сессионный идентификатор, который хранится у клиента и переиспользуется в запросах для идентификации) 
   - Этот токен/сессионный идентификатор можно использовать в последующих запросах

**Загрузка файлов**
   - Некоторые сайты позволяют загружать изображения, документы, аудио и видео через `POST`

---

__Чем отличается передача данных в POST-запросе от параметров в GET-запросе__:

**GET**-запрос  
   - Данные передаются в **URL** в виде параметров (`?key=value&key2=value2`)
   - Используется для **получения** информации (например, поиск в Google / поиск с фильтрами на сайте)
   - Параметры видны в браузере и могут быть закэшированы

**POST**-запрос  
   - Данные передаются **в теле запроса** (не в URL).  
   - Используется для **отправки** данных (авторизация, отправка форм, загрузка файлов).  
   - Данные **не видны в URL**, что делает его более безопасным

**Примечание**

Если бы чувствительные данные (например, логин и пароль) передавались в URL, появляются следующие проблемы:
- URL-адреса видны в логах серверов/прокси
- URL-адреса видны в истории браузера
- URL-адреса могут отсвечиваться в рефёрерах (когда вы переходите по внешней ссылке с сайта *example.com*, браузер может подставить в параметр `Referer` заголовка запроса что-то типа *example.com/catalogue?log=login&pass=password* )

# Переходим к использованию API

**Напоминание**

**API** (Application Programming Interface) — это интерфейс, который позволяет программам взаимодействовать между собой. Он предоставляет набор функций, с помощью которых можно запрашивать данные или управлять внешними сервисами

Примеры API:
- Прогноз погоды ([OpenWeather API](https://openweathermap.org/appid))
- Работа с музыкой ([Spotify API](https://developer.spotify.com/))
- Доступ к фильмам ([Кинопоиск API](https://api.kinopoisk.dev/documentation))
- etc.
    

Очень часто API разрабатываются в стиле **REST**

**REST** (Representational State Transfer) — это архитектурный стиль API, который работает через HTTP

Короткая справка:
- [Здесь](https://realpython.com/api-integration-in-python/#rest-and-python-building-apis) (до "REST and Python: Building APIs" не включительно)
- или [Здесь](https://blog.skillfactory.ru/glossary/rest-api/)

Из ограничений/принципов, заложенных в **REST** нам особенно интересны следующие (скопировано из второй ссылки для удобства восприятия):

- Отсутствие записи состояния клиента (**Stateless**). Сервер не должен хранить информацию о состоянии (проведенных операций) клиента. Каждый запрос от клиента должен содержать только ту информацию, которая нужна для получения данных от сервера.

- Единообразие интерфейса Rest (**Uniform Interface**). Все данные должны запрашиваться через один URL-адрес стандартными протоколами, например, HTTP. Это упрощает архитектуру сайта или приложения и делает взаимодействие с сервером понятнее

И ещё один важный термин - эндпоинты. **Эндпоинты** — это URL-адреса, через которые можно взаимодействовать с API (по сути это адреса конкретных ресурсов сервера / разделов сайта)

Пример:

- https://api.example.com/users  
- https://api.example.com/products/123

**Важно**

При веб-скрэппинге мы сталкиваемся с двумя ситуациями, работая со сторонним API:
- Либо мы (например, через инспектор [панель инструментов разработчика в веб-браузере]) пытаемся понять логику отправки запросов браузера серверу и воспроизвести её сами
- Либо мы взаимодействуем с API, используя документацию, и здесь мы как правило должны пройти авторизацию

<img src="auth_meme.jpg" width=400 height=500 title="демотиватор 2">

**Аутентификация** — это процесс подтверждения вашей личности, например, при входе в систему с помощью логина и пароля. Цель — убедиться, что вы тот, за кого себя выдаете (типа "проверка аутентичности" ~ authentication)

**Авторизация** — это процесс предоставления вам прав доступа к определенным ресурсам или действиям после того, как ваша личность была подтверждена (например, доступ к просмотру или редактированию данных). Т.е. "авторизироваться на сайте" = получить определенные права на сайте

# Методы аутентификации

| Метод | Что это и как работает | Где используется | Примеры сервисов | Плюсы | Минусы |
|--------|------------------------|------------------|------------------|--------|--------|
| **Session-based аутентификация** | При входе на сервере создается сессия (отправляется POST-запрос с credentials в его теле), а клиент получает `session_id` в составе cookies в ответ. При каждом запросе сервер проверяет эту сессию | Веб-приложения с авторизацией | Wordpress, интернет-магазины | Удобно для пользователей, так как позволяет избежать необходимости авторизовываться при каждом запросе ресурса | Не подходит для API*, требует хранения сессий на сервере |
| **API-ключи** | Клиент получает уникальный ключ (alphanumeric string) и передает его в заголовке запроса или в URL. Такой ключ генерируется как правило один раз при регистрации на сервисе, как правило не имеет срока действия и используется для предоставления общего доступа к ресурсу (т.е. не содержит в себе **scope**) | Открытые API, публичные сервисы | OpenWeather, NASA API, Кинопоиск | Простая реализация, удобство в использовании | Если передавать в URL, можно легко перехватить (например, через реферер)* |
| **Basic-аутентификация** | Логин и пароль кодируются в Base64 (бинарные данные кодируются с помощью 64 ascii-символов) и передаются в заголовке `Authorization: Basic <encoded_credentials>`. | Закрытые API, внутренние сервисы | Примеры в документации REST API, старые API-интерфейсы | Простота использования (не требует сложных механизмов) | Низкая безопасность (логин и пароль могут быть украдены, даже в Base64) |
| **Bearer-аутентификация** | Клиент получает `access_token`, который передается в заголовке `Authorization: Bearer <token>`. Токен действует ограниченное время | API, требующие аутентификации | Spotify API, GitHub API| Безопаснее, чем логин и пароль, можно легко отозвать токен | Токен истекает, его нужно обновлять |
| **OAuth 2.0 (Open Authorization)** | Клиент получает `access_token` через процесс авторизации (например, через редиректы и выдачу разрешений пользователем). | API от имени пользователя | Google API, Facebook API, Twitter API, Spotify API | Позволяет давать доступ к сервису без передачи пароля | Сложен в реализации, требует дополнительных шагов |

**Справка**

1) Session-based аутентификация не подходит для API в основном по следующим причинам:
- API (особенно RESTful) по своей сути без состояния (stateless). Это означает, что каждый запрос к серверу должен **уже** содержать всю необходимую информацию для обработки, включая информацию о пользователе, его аутентификации и авторизации. Этот принцип (в REST) помогает хранить гораздо меньше информации о пользователе и облегчает масштабируемость
- Куки привязаны к конкретной пользовательской сессии на конкретном устройстве (в то время как ключ/токен можно выдать что называется platform-agnostic)
2) Из ограничений API-ключей:
- Ключ генерируется один раз и не истекает --> проблемы с безопасностью
- Ключ не содержит информации о правах доступа конкретного пользователя
- Часто процесс генерации происходит через веб-интерфейс (т.е. закодить сложнее)

- [Про методы аутентификации в целом](https://starkovden.github.io/authentication-and-authorization.html)
- [Про session-based, т.к. в верхней ссылке этого нет](https://www.geeksforgeeks.org/session-vs-token-based-authentication/)
- [Про OAuth 2.0 можно почитать здесь](https://skillbox.ru/media/code/chto-takoe-oauth-20-ponyatie-i-printsip-raboty/), но мы обсудим на семинаре

#### Пример 3: POST-запрос (для авторизации): basic authentication + bearer token

In [130]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [135]:
# пример отправки POST-запроса на получение токена для спотифая 
# сначала нужно получить токен, который будет прикрепляться к заголовкам запроса
# процесс получения токена происходит через механизм basic authentication

import requests
import base64 


# Укажите свой client_id и client_secret (берутся в веб-версии при создании приложения здесь https://developer.spotify.com/dashboard) 

client_id = os.getenv('client_id')  # аналог логина
client_secret = os.getenv('client_secret')  # аналог пароля
auth_url = 'https://accounts.spotify.com/api/token'

raw_credentials = f"{client_id}:{client_secret}"
print(f"Данные без кодирования: {client_id[:4]}...:{client_secret[:4]}...")
# мы вешаем .encode() на строку, поскольку перед кодированием в base64 данные должны быть представлены в бинарном виде
b64_decoded = base64.b64encode(raw_credentials.encode()).decode("ascii")
print(f"Закодированные данные в base64 (в формате ascii, ибо это часть стандарта HTTP): {b64_decoded[:5]}...")

# ------
# Далее формируем запрос
headers = {
    'Authorization': 'Basic ' + b64_decoded
}

data = {
    'grant_type': 'client_credentials',
}

response = requests.post(url=auth_url, 
                         headers=headers, 
                         data=data)

if "access_token" in response.json():
    token = response.json()['access_token']
    print(f"Access Token: {token[:7]}...")
else:
    print("Токен не был получен")
    token = ""


# -------
# Далее используем данный токен при отправке запросов к API эндпоинтам

url = "https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg"
bearer_header = {"Authorization": "Bearer " + token}
response = requests.get(url=url,
                        headers=bearer_header
                       )

print("\n\n")
print(response.text)

Данные без кодирования: c738...:1c6c...
Закодированные данные в base64 (в формате ascii, ибо это часть стандарта HTTP): YzczO...
Access Token: BQDCi8I...



{"external_urls":{"spotify":"https://open.spotify.com/artist/0TnOYISbd1XYRBk9myaseg"},"followers":{"href":null,"total":11340013},"genres":[],"href":"https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg","id":"0TnOYISbd1XYRBk9myaseg","images":[{"url":"https://i.scdn.co/image/ab6761610000e5eb4051627b19277613e0e62a34","height":640,"width":640},{"url":"https://i.scdn.co/image/ab676161000051744051627b19277613e0e62a34","height":320,"width":320},{"url":"https://i.scdn.co/image/ab6761610000f1784051627b19277613e0e62a34","height":160,"width":160}],"name":"Pitbull","popularity":85,"type":"artist","uri":"spotify:artist:0TnOYISbd1XYRBk9myaseg"}


# Введение в Selenium

__Почему requests + BeautifulSoup не всегда работают?__
- **Requests** и **BeautifulSoup** — отличные инструменты для парсинга статических HTML-страниц. Однако они не могут работать с контентом, который генерируется динамически с помощью JavaScript
- Например, если сайт загружает данные с помощью JavaScript после первоначальной загрузки страницы, запрос через `requests` не получит нужный контент (requests не выполняет JavaScript-код)
- В таких случаях для парсинга (и скрэппинга перед ним) нужно использовать инструменты, которые могут взаимодействовать с браузером, такие как **Selenium** 

__Что такое Selenium?__
- [**Selenium**](https://www.selenium.dev/) — это библиотека для автоматизации браузеров (открытие страниц, клики, ввод текста, скроллы и т.д.)
- Selenium может управлять реальными браузерами, такими как Chrome, Firefox, и другие, что позволяет работать с динамическим контентом, который генерируется с помощью JavaScript

__Как работает WebDriver (Chromedriver, Geckodriver)__
- **WebDriver** — это интерфейс для взаимодействия с браузером.
- **Chromedriver** — это инструмент, который позволяет Selenium управлять браузером Google Chrome.
- **Geckodriver** — аналог для браузера Mozilla Firefox.
- Эти драйвера необходимы для того, чтобы Selenium мог взаимодействовать с конкретным браузером.

In [ ]:
!pip install selenium

In [165]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [166]:
driver = webdriver.Chrome()

driver.get("https://www.selenium.dev/selenium/web/web-form.html")

In [167]:
driver.get("https://student.itmo.ru/ru/dormitory/")

In [170]:
dorm_abouts = driver.find_elements(by=By.CSS_SELECTOR, value=".card__info-link.link-arrow.icon-arrow-right")

In [173]:
import time

time.sleep(2)
dorm_abouts[0].click()

In [178]:
driver.quit()  # любые попытки взаимодействия с драйвером приведут к выбросу ошибки

# Что почитать/посмотреть

__Общее__
- [Про HTTP запросы](https://habr.com/ru/articles/865040/) (дублирую из первого семинара, если ещё не читали)
- [Этот ридинг](https://realpython.com/api-integration-in-python/#rest-and-python-building-apis) (до "REST and Python: Building APIs" не включительно). Здесь во многом идёт пересечение с самым первым общим ридингом, но конкретные запросы (GET/POST/etc.) рассматриваются более предметно

__requests__
- [Про отправку запросов с использованием объекта `requests.Session()`](https://docs-python.ru/packages/modul-requests-python/sessii-seansy-session/)
- [Здесь же ссылка на доку по `Session()`, если надо (не рекомендую читать как гайд, это не гайд)](https://requests.readthedocs.io/en/latest/user/advanced/)

__REST API__
- [Здесь говорится про REST API отдельно](https://blog.skillfactory.ru/glossary/rest-api/). Полезно почитать просто с целью понять, почему современные API строятся так, как они строятся (идейно)

__Аутентификация__
- [Про методы аутентификации](https://starkovden.github.io/authentication-and-authorization.html). Отсюда почитайте до Basic Authentication включительно. Вы можете почитать про HMAC ради интереса, но нам такой способ вряд ли встретится (про OAuth 2.0 пока читать не надо, если нет личного желания)
- [Про аутентификацию на основе сессий](https://habr.com/ru/companies/vk/articles/343288#autentifikaciya-na-osnove-sessiy). Ссылка ведёт на конкретный абзац текста (я сейчас не советую читать статью целиком). В этой же статьей можно прочитать про [аутентификацию на основе токенов](https://habr.com/ru/companies/vk/articles/343288/#autentifikaciya-na-osnove-tokenov) (Bearer authentication в таблице сверху - это частный случай такой аутентификации)

__Selenium__
- [Официальный мини-гайд](https://www.selenium.dev/documentation/webdriver/getting_started/)
- [Документация](https://selenium-python.readthedocs.io/). Полезно, если нужно узнать, как повзаимодействовать с каким-то специфичным элементом

__Видосики__
- [Вот этот плейлист про скрэппинг статических страниц](https://www.youtube.com/playlist?list=PLRzwgpycm-Fio7EyivRKOBN4D3tfQ_rpu)
- [Вот этот плейлист про скрэппинг динамического контента](https://www.youtube.com/playlist?list=PLRzwgpycm-FgQ9lP_JTfrCa9O573XiJph)